# Sam, find exact line of code where the populations are specified

### Import Statements

In [0]:
%pip install /dbfs/FileStore/jars/82a8f80e_8266_43fb_be3c_12ac7a64dc2d/maas_utils-0.2.0-py3-none-any.whl
%pip install h2o==3.36.0.4
%pip install mlflow==1.18.0
%pip install bayesian-optimization
%pip install seaborn==0.11.0
%pip install xgboost
%pip install shap
%pip install h2o-pysparkling-3.2==3.36.0.4.post1
%pip install matplotlib>=3.2.0
%pip install eli5

%pip install hyperopt
%pip install mlflow==1.24.0
%pip install faker
%pip install /dbfs/FileStore/jars/82a8f80e_8266_43fb_be3c_12ac7a64dc2d/maas_utils-0.2.0-py3-none-any.whl


In [0]:
%run /MDA/MODELING_DEV/segmentation/iteration1/Segmentation-prospect-legacy-customer/helper_functions

In [0]:
#Import statements an d initial table 
from mass_utils import pyspark_helpers as pysh
from mass_utils import databricks_helpers as dbh
import pyspark.pandas as ps 
import seaborn as sns
from sklearn.linear_model import LinearRegression
#Set a larger seaborn figure size for display 
sns.set(rc={'figure.figsize':(11.7,8.27)})
from sklearn.metrics import r2_score
import numpy as np
import databricks.koalas as ks
pd.set_option('display.max_rows', 700)
import h2o 
from mass_utils.pandas_helpers import max_replace
from mass_utils.pandas_helpers import zero_replace
#from pysparkling import *
from h2o.automl import H2OAutoML
import mlflow
pd.options.display.max_rows = 1000
#pd.set_option('display.max_colwidth', None)
from h2o.estimators import H2ORuleFitEstimator
import datetime as dt
# from pandasql import sqldf
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch
from bayes_opt import BayesianOptimization
import tempfile
pysqldf = lambda q: sqldf(q, globals())
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
import mlflow
from mlflow.tracking import MlflowClient

In [0]:
from pysparkling import *

In [0]:
## Can you confirm these have been replaced correctly?? 

# train_dataset = "prospect_legacy_current_2019_10_15_2020_01_01_2020_04_15_2020_06_20_training"
# train_preprocessed_dataset = "prospect_legacy_current_2019_10_15_2020_01_01_2020_04_15_2020_06_20_processed"
# score_dataset = "prospect_legacy_current_2021_06_01_scoring"
# score_preprocessed_dataset = "prospect_legacy_current_2021_06_01_processed"

In [0]:
dbutils.widgets.text('training_experiment_path', 'experiment_path')
dbutils.widgets.text('scoring_experiment_path', 'experiment_path')
# dbutils.widgets.text('population', 'land customer')
dbutils.widgets.text('scoring_raw_dataset', 'dataset')
dbutils.widgets.text('scoring_preprocessed_dataset', 'dataset')
dbutils.widgets.text('score_date', '2021-09-26')

# dbutils.widgets.removeAll()


In [0]:
train_exp_path = getArgument('training_experiment_path')
score_exp_path = getArgument('scoring_experiment_path')
pop = 'Land Customer' 
score_preprocessed_dataset = getArgument('scoring_preprocessed_dataset')
scoring_raw_dataset = getArgument('scoring_raw_dataset')
score_date = getArgument('score_date')

In [0]:
print("Train Experiment Path = ", train_exp_path)
print("Scoring Experiment Path = ", score_exp_path)
print("Population = ", pop)
print("Score data table name = ", score_preprocessed_dataset)
print("Score raw data table name = ", scoring_raw_dataset)
print("Score Date = ", score_date)

In [0]:
client = MlflowClient()

train_exp = mlflow.get_experiment_by_name(train_exp_path)
train_exp_id = train_exp.experiment_id

display(client.list_run_infos(train_exp_id))

In [0]:
mlflow_query = "tags.active = 1 and tags.account_population = '" + pop + "'"
active_run = client.search_runs(experiment_ids=train_exp_id, filter_string=mlflow_query)[0].to_dictionary()
artifact_uri = active_run['info']['artifact_uri'] + '/'
run_id = active_run['info']['run_id']

In [0]:
artifact_objects = client.list_artifacts(run_id)
artifacts_paths = {}
for item in artifact_objects:
  if 'dataset' in item.path:
    artifacts_paths['train_dataset'] = item.path
  if 'model_python' in item.path:
    artifacts_paths['model_path'] = item.path
  if 'features_only' in item.path:
    artifacts_paths['features'] = item.path
  if 'reqs' in item.path:
    artifacts_paths['requirements'] = item.path

artifacts_paths

In [0]:
h2o.init()
saved_model = mlflow.h2o.load_model(artifact_uri + artifacts_paths['model_path'])

In [0]:
# Download artifact: training dataset, features list
import os
client = MlflowClient()
local_dir = "/tmp/artifact_downloads/segmentation/land_customer"
if not os.path.exists(local_dir):
    os.makedirs(local_dir)
local_path = client.download_artifacts(run_id, artifacts_paths['train_dataset'], local_dir)
print("Artifacts downloaded in: {}".format(local_dir))
print("Artifacts: {}".format(os.listdir(local_dir)))
train_pd = pd.read_csv(local_dir+'/'+artifacts_paths['train_dataset'])

In [0]:
df = spark.sql('select * from '+ score_preprocessed_dataset + '').toPandas()
print(df.shape)

In [0]:
%sql
select case when last_3years_booking_amount=0 then 'Legacy'
            when last_3years_booking_amount is null then 'Prospect'
            else 'current'
            end as acct_status, count(case when last_3years_booking_amount=0 then 'Legacy'
            when last_3years_booking_amount is null then 'Prospect'
            else 'current'
            end)
from seg_whitespace_raw_dataset_test
group by (case when last_3years_booking_amount=0 then 'Legacy'
            when last_3years_booking_amount is null then 'Prospect'
            else 'current'
            end)

In [0]:
import numpy as np
#df['randNumCol'] = np.random.randint(1, 100, df.shape[0])
df['calendar_date'] = score_date
df['calendar_date_str'] = df['calendar_date'].str.replace('-', '')

In [0]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator
h2o.init()

hc = H2OContext.getOrCreate()

In [0]:
### Grab features 
local_path = client.download_artifacts(run_id, artifacts_paths['features'], local_dir)
print("Artifacts downloaded in: {}".format(local_dir))
print("Artifacts: {}".format(os.listdir(local_dir)))

features_txt = open(local_path, "r").read()
feature_list = features_txt.replace('"','').replace("[","").replace("]","").replace("'","").split(", ")

print("Features: ", feature_list)

In [0]:
X_new=df[feature_list]
y=df['next_year_pi_acv_with_multiplier'].tolist()
train = X_new
train["dv"] = y
h2o_df = h2o.H2OFrame(train)
h2o_df['calendar_date_str'] = h2o_df['calendar_date_str'].asfactor()

In [0]:
#Score the training data
predict = saved_model.predict(h2o_df)
#Add the prediction column to the origna H2O frame
h2o_df['prediction'] = predict
#Convert to Pandas
scored_df = h2o_df.as_data_frame()
accounts = df['account_id'].tolist()
scored_df['account_id'] = accounts

In [0]:
scored_spark_df = spark.createDataFrame(scored_df)
#Create SQL temp view
scored_spark_df.createOrReplaceTempView("scored_dataset")

### Score Accuracy Metrics

In [0]:
import numpy as np 
import sklearn.metrics as metrics

y = scored_df['dv']
yhat = scored_df['prediction']

mae_score = metrics.mean_absolute_error(y, yhat)
mse_score = metrics.mean_squared_error(y, yhat)
rmse_score = np.sqrt(mse_score) # or mse**(0.5)  
r2_scored = metrics.r2_score(y,yhat)

print("Results of sklearn.metrics:")
print("MAE:",mae_score)
print("MSE:", mse_score)
print("RMSE:", rmse_score)
print("R-Squared:", r2_scored)

### Filter to only Land Customers for Analysis

In [0]:
### Is this join to get at the field:  last_3years_booking_amount? The table that does have it is hard coded so not sure how to recreate it, so just using the preprocessed table instead

view_name = "land_scored"
stmt = """create or replace temp view """ + view_name + """ as (
          select a.*, case when b.last_3years_booking_amount == 0 then 'Legacy Customer' 
                 when b.last_3years_booking_amount > 0 then 'Current Customer'
                 else 'Prospect Customer'
                 end as acct_status_code
          from scored_dataset a inner join """ +scoring_raw_dataset+""" b on a.account_id=b.account_id
          where (case when b.last_3years_booking_amount == 0 then 'Legacy Customer' 
                 when b.last_3years_booking_amount > 0 then 'Current Customer'
                 else 'Prospect Customer'
                 end) in ('Prospect Customer','Legacy Customer')

            ) """
spark.sql(stmt)

In [0]:
%sql
select acct_status_code, count(acct_status_code)
from land_scored
group by acct_status_code

In [0]:
scored_df = spark.sql('select * from land_scored').toPandas()

In [0]:
%sql
select account_id
from land_scored 
where account_id in ('0011T00002fHdGhQAK')

### Overview

In [0]:
# create dv and predict bins
bins=[float('-inf'),5000,10000,100000,float('inf')]
bin_labels=['1.non_spend','2.low_spend','3.med_spend','4.high_spend']
scored_df['spend_bins']=pd.cut(x=scored_df.dv, right=True,include_lowest=True, bins=bins,labels=bin_labels).astype(str)


scored_df['predict_spend_bins']=pd.cut(x=scored_df.prediction, right=True,include_lowest=True, bins=bins,labels=bin_labels).astype(str)
pct_error_bins=[float('-inf'),.05,.1,.2,.3,.4, .5,.6,.7,.8,.9,1,float('inf')]


scored_df['abs_residual'] = abs(scored_df['dv'] - scored_df['prediction'])
scored_df['err_is_X_times_spend'] = np.where(scored_df['dv']>0, scored_df['abs_residual'] / scored_df['dv'], np.nan)
scored_df['pct_error_bins']=pd.cut(x=scored_df.err_is_X_times_spend, right=True,include_lowest=True,bins=pct_error_bins).astype(str)

In [0]:
#box_fig = scored_df.boxplot(column=['dv','prediction'])
#box_fig_name = '/dbfs/FileStore/land_boxplot_dv_pred.png'
#box_fig.figure.savefig(box_fig_name)

In [0]:
# r2 reps degree that adjusted_predicted & actual values are correlated
## R2 for each groupings
import json
from sklearn.metrics import r2_score
all_features = feature_list+ ['dv']
score_h2o= h2o.H2OFrame(scored_df[all_features])
score_h2o['calendar_date_str'] = score_h2o['calendar_date_str'].asfactor()
perf_score = saved_model.model_performance(score_h2o[all_features])

r2_dict = {}
print("Original General r2: ", perf_score.r2())

n = scored_df.shape[0]
p = scored_df.shape[1]
r2 = r2_score(scored_df['dv'], scored_df['prediction'])
adjusted_r2 = 1 - ((1-r2) * ((n - 1) / (n - p - 1)))
r2_dict['General'] = r2


print("New General r2: ",r2)
print("New General Adjusted r2: ",adjusted_r2)
print()

r2_file_name = '/dbfs/FileStore/land_r2_land_groupings.json'
with open(r2_file_name, 'w') as outfile: 
  json.dump(r2_dict, outfile)  

In [0]:
y_actual = scored_df['spend_bins']
y_pred = scored_df['predict_spend_bins']

df_conf = pd.crosstab(y_pred,y_actual)

# save as csv
csv = '.csv'
conf_matrix_file = '/tmp/land_spend_conf_matrix' + csv
df_conf.to_csv(conf_matrix_file, index=False)

df_conf

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc={'figure.figsize':(11.7,8.27)})


ax = (scored_df[scored_df['dv'] >= 0]
.groupby('spend_bins')['pct_error_bins']
.value_counts(normalize=True)
.mul(100)
.rename('percent')
.reset_index()
.pipe((sns.catplot,'data'), x='spend_bins',y='percent',hue='pct_error_bins',kind='bar', palette='pastel',hue_order=sorted(scored_df['pct_error_bins'].unique()),aspect=5))

errors_graph_file_name = '/dbfs/FileStore/land_errors.png'
ax.savefig(errors_graph_file_name)

In [0]:
features_only = feature_list

In [0]:
### Save model parameters to json
import json
params = (saved_model.params)
params_name = '/dbfs/FileStore/land_model_params.json'
with open(params_name, 'w') as outfile:
  json.dump(params, outfile)
  
  
### Save features list to json
features_only_txt_name = '/dbfs/FileStore/land_features_only_list.txt'
with open(features_only_txt_name, 'w') as outfile: 
  json.dump(str(features_only), outfile)
  
  
### Save MAE per spend bin dict
spend_bins_mae_file = '/dbfs/FileStore/land_mae_bins.json'
mae_dict = scored_df.groupby('spend_bins')['abs_residual'].mean().to_dict()
with open(spend_bins_mae_file, 'w') as outfile: 
  json.dump(mae_dict, outfile)
  
### Save scored dataset
scored_file = '/dbfs/FileStore/land_scored_dataset.csv'
scored_df.to_csv(scored_file, index=False)
  
  
### Save README
read_me = "Stacked GBM Tweedie Without Constant (Upsell DV) on 2021-06-01"
read_me_txt_name = '/dbfs/FileStore/land_readme.txt'
with open(read_me_txt_name, 'w') as outfile: 
  json.dump(read_me, outfile)

In [0]:
%sh pip freeze > /dbfs/FileStore/land_reqs.txt

In [0]:
#Covert to spark
scored_spark_df = spark.createDataFrame(scored_df)
#Create SQL temp view
scored_spark_df.createOrReplaceTempView("scored_prospects_score")

### Adding Constant (Duan Smearing) And Re-running Analysis

In [0]:
scored_df = spark.sql('select * from scored_prospects_score').toPandas()

In [0]:
train_pd['abs_residual'] = abs(train_pd['dv'] - train_pd['prediction'])
train_pd['residual'] = train_pd['dv'] - train_pd['prediction']

In [0]:
# q_bins = [0,0.8,0.95, 0.9763,1]
q_bins = [0, 0.97, 0.98,1]
#q_bins = [0,0.9824,0.9917,0.9957,1]
train_quantiles_df = train_pd[['dv','residual','abs_residual']].quantile(q_bins).reset_index()
q_bins_spend = train_quantiles_df.dv.tolist()
display(train_quantiles_df)

In [0]:
# View quantiles and related spend
q_bins_spend[-1] = float('inf')
q_bins_spend[0] = 0
{q_bins[i]: q_bins_spend[i] for i in range(len(q_bins))}

In [0]:
train_pd['quantile_spend_bins'] = pd.cut(x=train_pd.dv, right=True,include_lowest=True,bins=q_bins_spend).astype(str)
train_pd.quantile_spend_bins.value_counts()

### Cal. Duan Smearing Constant

In [0]:
train_pd.abs_residual.mean()

In [0]:
# create dictionary where keys = interval and values = duan estimate
duan_estimate_df = train_pd.groupby('quantile_spend_bins').abs_residual.mean().to_frame()
duan_dict = duan_estimate_df.to_dict()['abs_residual']
smallest_bin = [col for col in list(duan_dict.keys()) if '-0.001' in col][0]
duan_dict[smallest_bin] = 0

duan_dict

### Apply duan estimate to each group respectively
- Don't correct 1st group (predictions -inf to X) bc don't want to overpredict non/low spenders
- Apply to rest of group

In [0]:
df_pd = spark.sql('select * from scored_prospects_score').toPandas()

In [0]:
df_pd['quantile_spend_bins'] = pd.cut(x=df_pd.prediction, right=True,include_lowest=True,bins=q_bins_spend).astype(str)
df_pd['duan_constant'] = df_pd['quantile_spend_bins'].apply(lambda x: duan_dict[x])
df_pd['duan_predict'] = df_pd.apply(lambda x: x['prediction'] + duan_dict[x['quantile_spend_bins']], axis=1)

In [0]:
df_pd.quantile_spend_bins.value_counts()

### Round Predictions

In [0]:
def round_nearest(x,cutoff=100000):
  '''Round to 0 or nearest 10K or 100K based on cutoff'''
  def round_nearest_multiple(number,multiple):
    '''Returns a float rounded to the nearest multiple'''
    return round(float(number)/multiple)*multiple
  def round_up_nearest_multiple(number,multiple):
    '''Returns a float rounded up to nearest multiple'''
    return math.ceil(float(number)/multiple)*multiple
  
  if x < 5000:
    return 0
  elif x < cutoff:
    if x <= 10000 and x > 5000:
      return round_up_nearest_multiple(x,1000) # round up to nearest 10K if between 5-10K
    return round_nearest_multiple(x,10000) # round to nearest 10K
  elif x >= cutoff:
    return round_nearest_multiple(x,100000) # round to nearest 10K    

In [0]:
import math
df_pd['duan_pred_ratio'] = np.where(df_pd['dv']>0, df_pd['duan_predict'] / df_pd['dv'], np.nan)
## Round $$$ to 0, nearest 10K, or nearest 100K
df_pd['rounded_spend'] = df_pd['dv'].apply(round_nearest)
df_pd['rounded_prediction'] = df_pd['prediction'].apply(round_nearest)
df_pd['rounded_abs_residual'] = abs(df_pd['rounded_spend'] - df_pd['rounded_prediction'])
df_pd['rounded_residual'] = df_pd['rounded_spend'] - df_pd['rounded_prediction']
df_pd['rounded_duan'] = df_pd['duan_predict'].apply(round_nearest)
df_pd['duan_rounded_abs_residual'] = abs(df_pd['rounded_spend'] - df_pd['rounded_duan'])

In [0]:
df_pd['calendar_date']=score_date

In [0]:
#Covert to spark
scored_spark_df = spark.createDataFrame(df_pd)
scored_spark_df.createOrReplaceTempView("scored_duan_land_cust")

In [0]:
scored_constant_df = spark.sql('select * from scored_duan_land_cust').toPandas()

In [0]:
scored_df = scored_constant_df.copy()

In [0]:
import numpy as np 
import sklearn.metrics as metrics

y = scored_constant_df['rounded_spend']
yhat = scored_constant_df['rounded_prediction']

mae_score = metrics.mean_absolute_error(y, yhat)
mse_score = metrics.mean_squared_error(y, yhat)
rmse_score = np.sqrt(mse_score) # or mse**(0.5)  
r2_score = metrics.r2_score(y,yhat)

print("Results of sklearn.metrics:")
print("MAE:",mae_score)
print("MSE:", mse_score)
print("RMSE:", rmse_score)
print("R-Squared:", r2_score)

In [0]:
# create dv and predict bins
bins=[float('-inf'),5000,10000,100000,float('inf')]
bin_labels=['1.non_spend','2.low_spend','3.med_spend','4.high_spend']
scored_df['spend_bins']=pd.cut(x=scored_df.rounded_spend, right=True,include_lowest=True, bins=bins,labels=bin_labels).astype(str)


scored_df['predict_spend_bins']=pd.cut(x=scored_df.rounded_prediction, right=True,include_lowest=True, bins=bins,labels=bin_labels).astype(str)
pct_error_bins=[float('-inf'),.05,.1,.2,.3,.4, .5,.6,.7,.8,.9,1,float('inf')]


#df_pd['abs_residual'] = abs(df_pd['dv'] - df_pd['predict'])
scored_df['err_is_X_times_spend'] = np.where(scored_df['rounded_spend']>0, scored_df['rounded_abs_residual'] / scored_df['rounded_spend'], np.nan)
scored_df['pct_error_bins']=pd.cut(x=scored_df.err_is_X_times_spend, right=True,include_lowest=True,bins=pct_error_bins).astype(str)

In [0]:
box_fig = scored_df.boxplot(column=['rounded_spend','rounded_prediction'])
box_fig_name = '/dbfs/FileStore/land_boxplot_dv_pred.png'
box_fig.figure.savefig(box_fig_name)

In [0]:
y_actual = scored_df['spend_bins']
y_pred = scored_df['predict_spend_bins']

df_conf = pd.crosstab(y_pred,y_actual)

# save as csv
csv = '.csv'
conf_matrix_file = '/tmp/land_spend_conf_matrix' + csv
df_conf.to_csv(conf_matrix_file, index=False)

df_conf

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc={'figure.figsize':(11.7,8.27)})


ax = (scored_df[scored_df['rounded_prediction'] >= 0]
.groupby('spend_bins')['pct_error_bins']
.value_counts(normalize=True)
.mul(100)
.rename('percent')
.reset_index()
.pipe((sns.catplot,'data'), x='spend_bins',y='percent',hue='pct_error_bins',kind='bar', palette='pastel',hue_order=sorted(scored_df['pct_error_bins'].unique()),aspect=5))

errors_graph_file_name = '/dbfs/FileStore/land_errors.png'
ax.savefig(errors_graph_file_name)

In [0]:
ax = (scored_df[scored_df['rounded_prediction'] >= 1000]
.groupby('spend_bins')['predict_spend_bins']
.value_counts(normalize=True)
.mul(100)
.rename('percent')
.reset_index()
.pipe((sns.catplot,'data'), x='spend_bins',y='percent',hue='predict_spend_bins',kind='bar', palette='pastel',hue_order=sorted(scored_df['predict_spend_bins'].unique()),aspect=5))


spend_bins_graph_file_name = '/dbfs/FileStore/land_spend_bins.png'
ax.savefig(spend_bins_graph_file_name)

#### Shap Importance

In [0]:
## FROM EDE PULL IN 
select_stmt = dbh.generate_simple_ede_query(cols='*', ede_db='exp_mktg', ede_schema='DATA_SCIENCE', ede_table='view_account', is_distinct=False)
print(select_stmt)
mintigo_dataset_name2 = 'view_account'
mintigo_dataset = dbh.read_ede_data(select_stmt)
#mintigo_dataset = pysh.safe_name(mintigo_dataset)
mintigo_dataset.createOrReplaceTempView(mintigo_dataset_name2)

In [0]:
contributions = saved_model.predict_contributions(h2o_df) #calculate shap values
contributions_dataframe = contributions.as_data_frame() #convert to DF
contributions_dataframe['acct_index']=contributions_dataframe.index
try:
  contributions_matrix = contributions_dataframe[list(set(features_only) - set(['calendar_date_str']) -set(['name']))].values #convert  array
except KeyError as e: 
  # get problem features and remove from shap
  error_message = str(e)
  problem_ftrs = (error_message[error_message.find("["):error_message.find("]")+1])
  problem_ftrs = ast.literal_eval(problem_ftrs)
  contributions_matrix = contributions_dataframe[list(set(features_only) - set(['calendar_date_str'] + problem_ftrs))].values #convert array

  
expected_value = contributions_matrix.min()

In [0]:
# # transpose individual shap importance
# feature_columns = list(set(features_only) - set(['calendar_date_str']) -set(['name']))

# shap_transpose=contributions_dataframe.melt(id_vars='acct_index')
# shap_transpose = shap_transpose[shap_transpose.variable!='BiasTerm']
# shap_transpose = shap_transpose[shap_transpose.variable.isin(feature_columns)]
# shap_transpose = shap_transpose[shap_transpose.value!=0]

# # get top pos and negative shap
# pos_shap_df = shap_transpose[shap_transpose.value>0].groupby('acct_index')['value'].apply(lambda x: x.nlargest(5,keep='last')).to_frame().reset_index().merge(shap_transpose, how='inner',on=['acct_index','value']) #acct_index, value
# neg_shap_df = shap_transpose[shap_transpose.value<0].groupby('acct_index')['value'].apply(lambda x: x.nsmallest(5,keep='last')).to_frame().reset_index().merge(shap_transpose, how='inner',on=['acct_index','value']) #acct_index, value

# # combine into one shap df and get account id
# shap_df = pd.concat([pos_shap_df,neg_shap_df])

# scored_df['acct_index'] = scored_df.index
# shap_df = shap_df.merge(scored_df[['account_id','acct_index']],how='inner',on=['acct_index']).drop(['level_1','acct_index'],axis=1)

# shap_df = shap_df.rename(columns={'variable':'col_name','value':'feature_importance_vals'})
# shap_df = shap_df[['account_id','col_name','feature_importance_vals']]

In [0]:
# transpose individual shap importance
feature_columns = list(set(features_only) - set(['calendar_date_str']) -set(['name']))

shap_transpose=contributions_dataframe.melt(id_vars='acct_index')
shap_transpose = shap_transpose[shap_transpose.variable!='BiasTerm']
shap_transpose = shap_transpose[shap_transpose.variable.isin(feature_columns)]
shap_transpose = shap_transpose[shap_transpose.value!=0]
# get top pos and negative shap
pos_shap_df = shap_transpose[shap_transpose.value>0].groupby('acct_index',group_keys=False).apply(pd.DataFrame.nlargest, n=5, columns='value')
neg_shap_df = shap_transpose[shap_transpose.value<0].groupby('acct_index',group_keys=False).apply(pd.DataFrame.nsmallest, n=5, columns='value')
# combine into one shap df and get account id
shap_df = pd.concat([pos_shap_df,neg_shap_df])
scored_df['acct_index'] = scored_df.index
shap_df = shap_df.merge(scored_df[['account_id','acct_index']],how='inner',on=['acct_index']).drop(['acct_index'],axis=1)

shap_df = shap_df.rename(columns={'variable':'col_name','value':'feature_importance_vals'})
shap_df = shap_df[['account_id','col_name','feature_importance_vals']]


In [0]:
# add shap importance to predictions dataframe
pos_msk = shap_df.feature_importance_vals > 0
pos_shap = shap_df[pos_msk].groupby('account_id').apply(lambda x: dict(zip(x['col_name'],x['feature_importance_vals'])))
pos_shap = pos_shap.rename('Features_Positively_Contributing_to_Model')

neg_msk = shap_df.feature_importance_vals < 0
neg_shap = shap_df[neg_msk].groupby('account_id').apply(lambda x: dict(zip(x['col_name'],x['feature_importance_vals'])))
neg_shap = neg_shap.rename('Features_Negatively_Contributing_to_Model')

shap_df_pd = scored_df.merge(pos_shap, left_on='account_id', right_index=True).merge(neg_shap, left_on='account_id', right_index=True)


In [0]:
shap_final_df = shap_df_pd[['account_id','Features_Positively_Contributing_to_Model', 'Features_Negatively_Contributing_to_Model']]
#Covert to spark
shap_spark_df = spark.createDataFrame(shap_final_df)
#Create SQL temp view
shap_spark_df.createOrReplaceTempView("shap_df")

In [0]:
%sql
create or replace temp view shap_view as 
(select a.*, b.name
from shap_df a left join view_account b on a.account_id=b.ID
where UPPER(b.name) not like '%MARKETPLACE%'
)

In [0]:
shap_df_pd = spark.sql('select * from shap_view').toPandas()
scored_df = scored_df.merge(shap_df_pd, on='account_id')

In [0]:
fig = saved_model.shap_summary_plot(h2o_df, top_n_features = 50)
shap_imp_ftrs_plot = '/dbfs/FileStore/seg_land_cust_shap_imp_ftrs_plot.png'
plt.savefig(shap_imp_ftrs_plot)


In [0]:
# shap_df = spark.createDataFrame(shap_df_pd)


shap_file = '/dbfs/FileStore/seg_land_cust_shap_dataset.csv'
shap_df.to_csv(shap_file, index=False)

#### Logging Metrics to MLFlow

In [0]:
box_fig = scored_df.boxplot(column=['prediction','duan_predict', 'rounded_prediction','rounded_duan'])

box_fig_file = '/dbfs/FileStore/seg_land_cust_prediction_boxplot.png'
box_fig.figure.savefig(box_fig_file)

In [0]:
#Covert to spark
scored_spark_df = spark.createDataFrame(scored_df)
#Create SQL temp view
scored_spark_df.createOrReplaceTempView("scored_data_view")

In [0]:
%sql
create or replace temp view final_view as 
(select a.*
from scored_data_view a left join view_account b on a.account_id=b.ID
where UPPER(b.name) not like '%MARKETPLACE%'
)

In [0]:
scored_df = spark.sql('select * from final_view').toPandas()

# create stakeholder related columns
scored_df['prediction_start_date'] = pd.to_datetime(scored_df['calendar_date'])
scored_df['prediction_end_date'] = scored_df['prediction_start_date'] + pd.DateOffset(years=1)
scored_df['next_year_piacv_prediction_amt'] = scored_df['rounded_duan']
scored_df['buying_group_code'] = 'Land'
scored_df['next_year_growth_ind'] = 0
scored_df = scored_df.rename(columns={'account_id':'sfdcf5_acct_id'})


In [0]:
scored_file = '/dbfs/FileStore/seg_land_cust_scored_dataset.csv'
scored_df.to_csv(scored_file, index=False)

In [0]:
max_prediction = scored_df.duan_predict.max()
max_prediction

In [0]:
bin_avg_pct_errors_file = '/dbfs/FileStore/seg_land_cust_train_duan_constant_mapping.json'
with open(bin_avg_pct_errors_file, 'w') as outfile: 
  json.dump(duan_dict, outfile)

In [0]:
kde_cols = ['prediction','duan_predict','rounded_prediction']
kde_df = scored_df[kde_cols]

In [0]:
train_dates = list(set(train_pd['calendar_date_str'].tolist()))

In [0]:
### FIX THE HARDCODED DATE HERE

ax = sns.kdeplot(data=scored_df[scored_df['prediction']>=100000], x='prediction', thresh=0.05, color='green',fill=True, common_grid=True, legend=True)

ax = sns.kdeplot(data=scored_df[scored_df['duan_predict']>=100000], x='duan_predict', thresh=0.05, color='yellow',fill=True, common_grid=True, legend=True)

ax = sns.kdeplot(data=train_pd[(train_pd['dv']>=100000) & (train_pd['calendar_date_str']==train_dates[-1]) ], x='dv', thresh=0.05, color='purple',fill=True, common_grid=True, legend=True)

ax.figure.legend(labels=['Prediction','Prediction + Constant','Train DV'])
ax.figure.suptitle('KDE $ >= 100k', fontsize=20)

kde_mill_plus_file = '/dbfs/FileStore/seg_land_cust_kde_spend_100k_or_more.png'
ax.figure.savefig(kde_mill_plus_file)

In [0]:
### FIX THE HARDCODED DATE HERE

ax = sns.kdeplot(data=scored_df[scored_df['prediction']<=100000], x='prediction', thresh=0.05, color='green',fill=True, common_grid=True, legend=True)

ax = sns.kdeplot(data=scored_df[scored_df['duan_predict']<=100000], x='duan_predict', thresh=0.05, color='yellow',fill=True, common_grid=True, legend=True)

ax = sns.kdeplot(data=train_pd[(train_pd['dv']<=100000) & (train_pd['calendar_date_str']==train_dates[-1]) ], x='dv', thresh=0.05, color='purple',fill=True, common_grid=True, legend=True)

ax.figure.legend(labels=['Prediction','Prediction + Constant','Train DV'])
ax.figure.suptitle('KDE $ <= 100k', fontsize=20)

kde_less_than_mill_file = '/dbfs/FileStore/seg_land_cust_kde_spend_100k_or_less.png'
ax.figure.savefig(kde_less_than_mill_file)

In [0]:
raw_df = spark.sql('select * from ' + scoring_raw_dataset)
ftr_summary_file = '/dbfs/FileStore/seg_land_cust_raw_data_ftrs_summary.csv'
raw_feature_list = list(set(feature_list) - set(['calendar_date_str']))
ftr_summary = raw_df.select(raw_feature_list).toPandas().describe(include='all').reset_index()

ftr_summary.to_csv(ftr_summary_file, index=False)

In [0]:
prediction_related_columns = [col for col in scored_df.columns if 'predict' in col]
predict_summary = scored_df[prediction_related_columns].describe(include='all').reset_index()

predict_summary_file = '/dbfs/FileStore/seg_land_cust_prediction_summary.csv'
predict_summary.to_csv(predict_summary_file)

predict_summary

#### MLFLow Logging Function

In [0]:
%run /MDA/MODELING_DEV/segmentation/iteration3/ITERATION2_20202021_train/mlflow-logging-functions

In [0]:
import os.path
model_name = os.path.basename(artifacts_paths['model_path'])

production_score_log(title = score_date + ' Production WhiteSpace Customer Scores'
                     ,  original_dv = 'next_year_pi_acv_with_multiplier'
                     , train_date = active_run['data']['tags']['train_date']
                     , score_date = score_date
#                      , model = saved_model
                     , model_path = model_name
                     , boxplot_graph = box_fig_file
                     , scored_data = scored_file
                     , shap_data = shap_file
                     , max_prediction = scored_df.duan_predict.max()
                     , constants_mapping = bin_avg_pct_errors_file
                     , account_population = pop
                     , kde_higher_spender_file = kde_mill_plus_file
                     , kde_lower_spender_file = kde_less_than_mill_file
                     , shap_imp_ftrs_plot = shap_imp_ftrs_plot 
                     , ftr_summary_file = ftr_summary_file
                     , predictions_summary_file = predict_summary_file
                     , exp=score_exp_path)